In [ ]:
import os

# os.chdir("../")
%pwd

: 

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path

In [3]:
from src.MLOps_project_dev_v1.constants import *
from src.MLOps_project_dev_v1.utils.common import read_yaml, create_directories 

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config



In [5]:
import os
import urllib.request as request
import zipfile
from src.MLOps_project_dev_v1 import logger
from src.MLOps_project_dev_v1.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
                )
            
            logger.info(f"{filename} downloaded! with the following header:{headers}")
        else:
            logger.info(f"file already exist of size :{get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)




In [11]:
try:
    congif = ConfigurationManager()
    data_ingestion_config = congif.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-09-10 14:01:19,405: INFO: common: yaml file:config\config.yaml loaded sucessfully]
[2023-09-10 14:01:19,410: INFO: common: yaml file:params.yaml loaded sucessfully]
[2023-09-10 14:01:19,410: INFO: common: yaml file:schema.yaml loaded sucessfully]
[2023-09-10 14:01:19,418: INFO: common: created directory at: artifacts]
[2023-09-10 14:01:19,418: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-10 14:01:20,422: INFO: 516376290: artifacts/data_ingestion/data.zip downloaded! with the following header:Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B330:32C7D4:1B7609:2966AF:64FD7ED7
Accept-Ranges: bytes
Date: Sun, 10 